**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.6MB/s 
     |████████████████████████████████| 51kB 4.3MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N82bc0b4c386941e5a7337d3784ce1e6d (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [3]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
print("RDFLib:")
for s,p,o in g:
  print(s,p,o)

print("SPARQL:")
myQuery = prepareQuery('''
  SELECT ?s
  WHERE { 
    ?s rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "rdfs": RDFS, "ns": ns}
)

for res in g.query(myQuery):
  print(res)

RDFLib:
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#UPM http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#University
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Jane
http://somewhere#Researcher http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#Sar

**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [4]:
print("RDFLib:")
for s,p,o in g:
  print(s)
for s,p,o in g:
  for s1,p1,o1 in g:
    print(s1)

print("SPARQL:")
myQuery = prepareQuery('''
  SELECT ?s
  WHERE { 
    { 
      ?s rdf:type ns:Person. 
    } UNION {
      ?subclass rdfs:subClassOf ns:Person.
      ?s rdf:type ?subclass
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)
for res in g.query(myQuery):
  print(res)

RDFLib:
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones
http://somewhere#UPM
http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#SaraJones
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#JaneSmith
http://somewhere#Researcher
http://somewhere#SaraJones
http://somewhere#JaneSmith
http://somewhere#University
http://somewhere#JohnSmith
http://somewhere#JaneSmith
http://somewhere#Researcher
http://somewhere#PhDstudent
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#PhDstudent
http://somewhere#JohnSmith
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family
http://somewhere#Person
http://somewhere#JohnSmith
http://somewhere#JaneSmith
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given
http://somewhere#SaraJones
http://somewhere#UPM
http://somewhere#Jo

**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [5]:
print("RDFLib")
for s,p,o in g:
  for s1,p1,o1 in g:
      print(s1,p1,o1)

for s,p,o in g:
  for s1,p1,o1 in g:
    for s2,p2,o2 in g:
      print(s2,p2,o2)

print("SPARQL")
myQuery = prepareQuery('''
  SELECT ?s ?o ?p
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?o ?p
    } UNION {
      ?s rdf:type ?s1.
      ?s ?o ?p.
      ?s1 rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "rdfs": RDFS,"rdf": RDF, "ns": ns}
)

for res in g.query(myQuery):
  print(res)

Se han truncado las últimas 5000 líneas del flujo de salida.
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
http://somewhere#PhDstudent http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Class
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Datatype
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN http://www.w3.org/2000/01/rdf-schema#range http://www.w3.org/2001/XMLSchema#string
http://somewhere#PhDstudent http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Researcher
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Property
http://somewhere#Person http://www.w3.org/1999/02/22-rdf-syntax-ns#t